# Data Scrubber

Load the CSV data from `../data/raw` and clean it for processing. Cleaned data will be stored in `../data/clean`.

TODO:

* Jan 2020: Still getting clearly erroneous time stamps that go backward in time. Need to filter these out.
* Jan 2020: I started receiving CSV data from Govee that contained extra columns. See `office_interior_export202001131740.csv`  :( 
* Jan 2020: I started receiving CSV data from Govee that contained non-physical humidity level changes. Need to filter these out.
* Mar 2020: I should also load the clean data and use that as the first filter for stuff I shouldn't load. That way I stop loading all the old files. Probably reverse iterate over the glob.

In [1]:
import os
import glob
import pandas as pd
import util
import datetime as dt
import numpy as np

In [2]:
cur_dir = os.getcwd()
exterior_files = glob.glob(cur_dir.replace('notebooks','data/raw/*exterior*'))
interior_files = glob.glob(cur_dir.replace('notebooks','data/raw/*interior*'))

In [3]:
govee_columns = ['Timestamp for sample frequency every 1 min', 
                 'Temperature_Celsius', 
                 'Relative_Humidity']

def combine_no_time_duplicates(files_glob):
    all_data = pd.DataFrame(columns=govee_columns)
    for f in files_glob:
        data = pd.read_csv(f, warn_bad_lines=True, error_bad_lines=False)
        data.set_index(keys=govee_columns[0])
        all_data = pd.concat([all_data, data], 
                             ignore_index=True,
                             join='inner',
                             copy=True,
                             sort=True)

    all_data[govee_columns[0]] = pd.to_datetime(all_data[govee_columns[0]], format="%Y-%m-%d %H:%M:%S", errors='coerce')
       
    return all_data

In [4]:
all_exterior_data = combine_no_time_duplicates(exterior_files)
all_exterior_data['location'] = util.Locations.EXTERIOR.value
# all_exterior_data.to_csv('../data/clean/exterior.data.csv')

all_interior_data = combine_no_time_duplicates(interior_files)
all_interior_data['location'] = util.Locations.INTERIOR.value
# all_interior_data.to_csv('../data/clean/interior.data.csv')


b'Skipping line 7137: expected 3 fields, saw 5\n'
b'Skipping line 20053: expected 3 fields, saw 5\nSkipping line 23465: expected 3 fields, saw 5\n'


In [5]:
# combine into one
all_data = pd.concat([all_exterior_data, all_interior_data])
all_data.rename(columns={govee_columns[0]: "timestamp", govee_columns[1]: str(govee_columns[1]).lower(), govee_columns[2]: str(govee_columns[2]).lower()},
                inplace=True)

I've noticed invalid data coming from `Govee` and this needs to be handled. First, get a view of the bad data. Anything prior to Nov 9, 2019 is before I owned the device so we know that data must be invalid.

What's up with this, `Govee`?

In [6]:
# invalidate any data points prior to a certain date/time
invalid_prior_to = dt.datetime(year=2019, month=11, day=9, hour=15, minute=0)
bad_data = all_data[all_data.timestamp < invalid_prior_to]
bad_data

,relative_humidity,temperature_celsius,timestamp,location
2130,77.7,4.3,2019-01-11 01:47:00,1
8153,51.6,-6.3,2019-11-01 04:26:00,2
24166,64.6,5.3,2019-11-02 09:53:00,2
52725,57.5,-1.1,2019-01-08 05:48:00,2
63792,62.2,5.6,2019-01-15 22:15:00,2


Now apply the timestamp filter to remove anamalous data.

In [7]:
strip_prior_to = dt.datetime(year=2019, month=11, day=9, hour=15, minute=30)
all_data = all_data[all_data.timestamp>=strip_prior_to]

In [8]:
timestamp_diff_threshold = dt.timedelta(minutes=5.0)
# above_threshold_series = all_data[govee_columns[0]].diff() > timestamp_diff_threshold 
# all_data.drop(above_threshold_series.index)


# Write Clean Data

In [9]:
all_data.to_csv('../data/clean/sensor_data.csv', index=False)

In [10]:
os.listdir('../data/clean/')

['sensor_data.csv']

In [11]:
all_data.head()

,relative_humidity,temperature_celsius,timestamp,location
30,36.4,7.6,2019-11-09 15:30:00,1
31,36.5,7.6,2019-11-09 15:31:00,1
32,36.8,7.6,2019-11-09 15:32:00,1
33,36.9,7.6,2019-11-09 15:33:00,1
34,37.6,7.6,2019-11-09 15:34:00,1
